<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=4a32035b1e64cb6572e1c3429ca61e95bc165fc4477b620946575192c7f91125
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta


In [2]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

In [3]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-5*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [4]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

mypf = mypf[mypf['InPortfolio'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()
stock_ids

Mounted at /content/drive


array(['5PAISA', 'AAVAS', 'AMARAJABAT', 'AMBUJACEM', 'ASIANPAINT',
       'BAJAJFINSV', 'BAJFINANCE', 'BANDHANBNK', 'BATAINDIA', 'BAYERCROP',
       'CLEAN', 'DMART', 'FINEORG', 'GICRE', 'GILLETTE', 'GLAND',
       'HDFCAMC', 'ICICIGI', 'INDIGOPNTS', 'INDUSINDBK', 'INFY', 'IRCTC',
       'ITBEES', 'JMFINANCIL', 'JUBLFOOD', 'LUXIND', 'NIACL', 'PAGEIND',
       'PIDILITIND', 'PNB', 'RELAXO', 'RELIANCE', 'ROSSARI', 'TCS',
       'TEAMLEASE', 'UJJIVANSFB', 'VAIBHAVGBL', 'VALIANTORG', 'WIPRO'],
      dtype=object)

In [5]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))
    stock_df = yfin.Ticker(stock_symbol).history(period='5y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Stock'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'
                   ], axis=1, inplace=True)

    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    #current_pe = get_current_pe(stock_id)
    stock_id = stock_id.upper() + '.NS'
    tmp = stock_prec_dev(stock_id)
    tmp['Prev_Close'] = tmp['Close'].shift(1)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Shares'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]
    tmp['Criteria'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Criteria'].values[0]
    tmp['Strategy'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Strategy'].values[0]
    #tmp['Curr_PE'] = current_pe
    df_prec_dev = df_prec_dev.append(tmp)

In [6]:
df_prec_dev.reset_index(drop=True, inplace=True)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

In [7]:
# portfolio stocks

tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df['Investment'] = tmp_df['Avg_Cost'] * tmp_df['Shares']
tmp_df['Current'] = tmp_df['Close'] * tmp_df['Shares']
tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
tmp_df['Estimated'] = tmp_df['Max'] * tmp_df['Shares']
tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
tmp_df['Estimated P/L%'] = round((tmp_df['Estimated'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
tmp_df['Juice Left%'] = tmp_df['Estimated P/L%'] - tmp_df['Current P/L%']

investment = round(sum(tmp_df['Avg_Cost']*tmp_df['Shares']),0)
current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
previous = round(sum(tmp_df['Prev_Close']*tmp_df['Shares']),0)

today_pnl_amount = current-previous
today_pnl_percentage = round((current-previous)*100/previous,2)

curr_pnl_amount = current-investment
curr_pnl_percentage = round((current-investment)*100/investment,2)

estimate = round(sum(tmp_df['Max']*tmp_df['Shares']),0)
est_pnl_amount = estimate-investment
est_pnl_percentage = round((estimate-investment)*100/investment,2)

tmp_df['Weightage%'] = round(tmp_df['Investment']*100/investment,2)
tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['Weightage%'],0)

print('Investment: ', investment)
print('Current: ', current)
print('Today PnL: ', today_pnl_amount)
print('Today PnL %: ', today_pnl_percentage)
print('Current PnL: ', curr_pnl_amount)
print('Current PnL %: ', curr_pnl_percentage)
print('Estimate: ', estimate)
print('Estimate PnL: ', est_pnl_amount)
print('Estimate PnL %: ', est_pnl_percentage)

qualified stocks: 39
Investment:  2773435.0
Current:  2833297.0
Today PnL:  -10037.0
Today PnL %:  -0.35
Current PnL:  59862.0
Current PnL %:  2.16
Estimate:  4990903.0
Estimate PnL:  2217468.0
Estimate PnL %:  79.95


In [8]:
tmp_df.sort_values(by = 'Current P/L%', ascending=False)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
35,38.10,UJJIVANSFB.NS,57.0,38.50,34.87,1350.0,V40N,ATH,47074.50,51435.00,51975.00,76950.0,-1.04,9.26,63.46,54.20,1.70,16.0
3,458.35,AMBUJACEM.NS,588.0,464.60,424.02,180.0,TBD,ATH,76323.60,82503.00,83628.00,105840.0,-1.35,8.10,38.67,30.57,2.75,22.0
6,7126.60,BAJFINANCE.NS,7901.0,7096.85,6602.15,24.0,V40,ATH,158451.60,171038.40,170324.40,189624.0,0.42,7.94,19.67,11.73,5.71,45.0
28,2680.70,PIDILITIND.NS,2902.0,2693.10,2504.06,30.0,V40,ATH,75121.80,80421.00,80793.00,87060.0,-0.46,7.05,15.89,8.84,2.71,19.0
11,3917.70,DMART.NS,5324.0,3704.85,3671.87,21.0,SS,ATH,77109.27,82271.70,77801.85,111804.0,5.75,6.69,44.99,38.30,2.78,19.0
4,3294.35,ASIANPAINT.NS,3542.0,3270.20,3091.34,25.0,V40,ATH,77283.50,82358.75,81755.00,88550.0,0.74,6.57,14.58,8.01,2.79,18.0
17,1209.50,ICICIGI.NS,1613.0,1212.05,1137.80,69.0,V40,ATH,78508.20,83455.50,83631.45,111297.0,-0.21,6.30,41.76,35.46,2.83,18.0
8,1614.60,BATAINDIA.NS,2173.0,1603.20,1523.62,100.0,V40,ATH,152362.00,161460.00,160320.00,217300.0,0.71,5.97,42.62,36.65,5.49,33.0
0,366.50,5PAISA.NS,553.0,365.95,347.06,228.0,V40N,ATH,79129.68,83562.00,83436.60,126084.0,0.15,5.60,59.34,53.74,2.85,16.0
24,497.00,JUBLFOOD.NS,903.0,494.30,471.56,164.0,SS,ATH,77335.84,81508.00,81065.20,148092.0,0.55,5.39,91.49,86.10,2.79,15.0


In [9]:
tmp_df.sort_values(by = 'Estimated P/L%', ascending=False)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
15,1008.45,GLAND.NS,4316.0,1001.10,1225.87,55.0,2T3Y,2T3Y,67422.85,55464.75,55060.50,237380.0,0.73,-17.74,252.08,269.82,2.43,-43.0
37,547.20,VALIANTORG.NS,1851.0,554.65,542.52,144.0,2T3Y,2T3Y,78122.88,78796.80,79869.60,266544.0,-1.34,0.86,241.19,240.33,2.82,2.0
25,1505.80,LUXIND.NS,4546.0,1506.40,1493.72,51.0,V40N,2T3Y,76179.72,76795.80,76826.40,231846.0,-0.04,0.81,204.34,203.53,2.75,2.0
36,305.30,VAIBHAVGBL.NS,966.0,307.90,317.48,144.0,2T3Y,2T3Y,45717.12,43963.20,44337.60,139104.0,-0.84,-3.84,204.27,208.11,1.65,-6.0
7,246.95,BANDHANBNK.NS,723.0,252.55,253.98,300.0,2T3Y,2T3Y,76194.00,74085.00,75765.00,216900.0,-2.22,-2.77,184.67,187.44,2.75,-8.0
26,120.15,NIACL.NS,317.0,120.30,117.07,390.0,TBD,ATH,45657.30,46858.50,46917.00,123630.0,-0.12,2.63,170.78,168.15,1.65,4.0
34,2339.15,TEAMLEASE.NS,5281.0,2356.80,2279.93,21.0,V40N,2T3Y,47878.53,49122.15,49492.80,110901.0,-0.75,2.60,131.63,129.03,1.73,4.0
1,1340.50,AAVAS.NS,3196.0,1350.40,1390.60,23.0,2T3Y,2T3Y,31983.80,30831.50,31059.20,73508.0,-0.73,-3.60,129.83,133.43,1.15,-4.0
18,1405.35,INDIGOPNTS.NS,3112.0,1421.80,1407.73,124.0,V40N,2T3Y,174558.52,174263.40,176303.20,385888.0,-1.16,-0.17,121.07,121.24,6.29,-1.0
13,181.65,GICRE.NS,358.0,182.85,174.18,231.0,2T3Y,2T3Y,40235.58,41961.15,42238.35,82698.0,-0.66,4.29,105.53,101.24,1.45,6.0


In [10]:
tmp_df.sort_values(by = 'Juice Left%', ascending=True)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
4,3294.35,ASIANPAINT.NS,3542.0,3270.20,3091.34,25.0,V40,ATH,77283.50,82358.75,81755.00,88550.0,0.74,6.57,14.58,8.01,2.79,18.0
28,2680.70,PIDILITIND.NS,2902.0,2693.10,2504.06,30.0,V40,ATH,75121.80,80421.00,80793.00,87060.0,-0.46,7.05,15.89,8.84,2.71,19.0
31,2552.55,RELIANCE.NS,2811.0,2551.85,2467.15,31.0,V40,ATH,76481.65,79129.05,79107.35,87141.0,0.03,3.46,13.94,10.48,2.76,10.0
6,7126.60,BAJFINANCE.NS,7901.0,7096.85,6602.15,24.0,V40,ATH,158451.60,171038.40,170324.40,189624.0,0.42,7.94,19.67,11.73,5.71,45.0
33,3216.30,TCS.NS,3857.0,3227.05,3231.00,16.0,V40,NaN,51696.00,51460.80,51632.80,61712.0,-0.33,-0.45,19.37,19.82,1.86,-1.0
5,1474.70,BAJAJFINSV.NS,1907.0,1480.40,1454.93,52.0,V40,ATH,75656.36,76684.40,76980.80,99164.0,-0.39,1.36,31.07,29.71,2.73,4.0
3,458.35,AMBUJACEM.NS,588.0,464.60,424.02,180.0,TBD,ATH,76323.60,82503.00,83628.00,105840.0,-1.35,8.10,38.67,30.57,2.75,22.0
22,30.09,ITBEES.NS,40.0,30.16,30.07,1675.0,SS,ATH,50367.25,50400.75,50518.00,67000.0,-0.23,0.07,33.02,32.95,1.82,0.0
17,1209.50,ICICIGI.NS,1613.0,1212.05,1137.80,69.0,V40,ATH,78508.20,83455.50,83631.45,111297.0,-0.21,6.30,41.76,35.46,2.83,18.0
8,1614.60,BATAINDIA.NS,2173.0,1603.20,1523.62,100.0,V40,ATH,152362.00,161460.00,160320.00,217300.0,0.71,5.97,42.62,36.65,5.49,33.0


In [11]:
tmp_df.sort_values(by = 'Weightage%', ascending=False)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
18,1405.35,INDIGOPNTS.NS,3112.0,1421.80,1407.73,124.0,V40N,2T3Y,174558.52,174263.40,176303.20,385888.0,-1.16,-0.17,121.07,121.24,6.29,-1.0
6,7126.60,BAJFINANCE.NS,7901.0,7096.85,6602.15,24.0,V40,ATH,158451.60,171038.40,170324.40,189624.0,0.42,7.94,19.67,11.73,5.71,45.0
8,1614.60,BATAINDIA.NS,2173.0,1603.20,1523.62,100.0,V40,ATH,152362.00,161460.00,160320.00,217300.0,0.71,5.97,42.62,36.65,5.49,33.0
27,38377.00,PAGEIND.NS,53549.0,38488.05,39325.03,3.0,V40,ATH,117975.09,115131.00,115464.15,160647.0,-0.29,-2.41,36.17,38.58,4.25,-10.0
14,4513.75,GILLETTE.NS,7529.0,4575.85,4475.85,21.0,V40,ATH,93992.85,94788.75,96092.85,158109.0,-1.36,0.85,68.21,67.36,3.39,3.0
0,366.50,5PAISA.NS,553.0,365.95,347.06,228.0,V40N,ATH,79129.68,83562.00,83436.60,126084.0,0.15,5.60,59.34,53.74,2.85,16.0
17,1209.50,ICICIGI.NS,1613.0,1212.05,1137.80,69.0,V40,ATH,78508.20,83455.50,83631.45,111297.0,-0.21,6.30,41.76,35.46,2.83,18.0
37,547.20,VALIANTORG.NS,1851.0,554.65,542.52,144.0,2T3Y,2T3Y,78122.88,78796.80,79869.60,266544.0,-1.34,0.86,241.19,240.33,2.82,2.0
4,3294.35,ASIANPAINT.NS,3542.0,3270.20,3091.34,25.0,V40,ATH,77283.50,82358.75,81755.00,88550.0,0.74,6.57,14.58,8.01,2.79,18.0
24,497.00,JUBLFOOD.NS,903.0,494.30,471.56,164.0,SS,ATH,77335.84,81508.00,81065.20,148092.0,0.55,5.39,91.49,86.10,2.79,15.0


In [12]:
tmp_df.to_csv('/content/drive/My Drive/data/stocks/myStocks-Portfolio-Analysis.csv', index=False)

In [13]:
tmp_df.sort_values(by = 'Risk Ind', ascending=True)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
15,1008.45,GLAND.NS,4316.0,1001.10,1225.87,55.0,2T3Y,2T3Y,67422.85,55464.75,55060.50,237380.0,0.73,-17.74,252.08,269.82,2.43,-43.0
27,38377.00,PAGEIND.NS,53549.0,38488.05,39325.03,3.0,V40,ATH,117975.09,115131.00,115464.15,160647.0,-0.29,-2.41,36.17,38.58,4.25,-10.0
7,246.95,BANDHANBNK.NS,723.0,252.55,253.98,300.0,2T3Y,2T3Y,76194.00,74085.00,75765.00,216900.0,-2.22,-2.77,184.67,187.44,2.75,-8.0
36,305.30,VAIBHAVGBL.NS,966.0,307.90,317.48,144.0,2T3Y,2T3Y,45717.12,43963.20,44337.60,139104.0,-0.84,-3.84,204.27,208.11,1.65,-6.0
1,1340.50,AAVAS.NS,3196.0,1350.40,1390.60,23.0,2T3Y,2T3Y,31983.80,30831.50,31059.20,73508.0,-0.73,-3.60,129.83,133.43,1.15,-4.0
10,1363.55,CLEAN.NS,2648.0,1368.45,1396.49,23.0,2T3Y,2T3Y,32119.27,31361.65,31474.35,60904.0,-0.36,-2.36,89.62,91.98,1.16,-3.0
19,1297.10,INDUSINDBK.NS,1965.0,1323.25,1301.47,36.0,SS,ATH,46852.92,46695.60,47637.00,70740.0,-1.98,-0.34,50.98,51.32,1.69,-1.0
33,3216.30,TCS.NS,3857.0,3227.05,3231.00,16.0,V40,NaN,51696.00,51460.80,51632.80,61712.0,-0.33,-0.45,19.37,19.82,1.86,-1.0
29,50.40,PNB.NS,96.0,51.80,50.85,900.0,SS,ATH,45765.00,45360.00,46620.00,86400.0,-2.70,-0.88,88.79,89.67,1.65,-1.0
9,4158.90,BAYERCROP.NS,6002.0,4216.65,4185.24,11.0,V40N,ATH,46037.64,45747.90,46383.15,66022.0,-1.37,-0.63,43.41,44.04,1.66,-1.0
